In [ ]:
#@title Google Drive Options { display-mode: "form" }
drive_mount = '/content/drive' #@param {type:"string"}
from google.colab import drive
drive.mount(drive_mount)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sentencepiece datasets

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).

 ## dataset

In [ ]:
from datasets import load_dataset
from tqdm.notebook import tqdm


ds_path="/content/drive/MyDrive/geniac_tokenizer/tokenizer_4category_train.jsonl"
output_file="/content/drive/MyDrive/geniac_tokenizer/merge_4category_tokenizer_4.0.txt"

ds_path="/content/drive/MyDrive/geniac_tokenizer/tokenizer_4category_val.jsonl"
output_file="/content/drive/MyDrive/geniac_tokenizer/merge_4category_tokenizer_4.0_val_one_third.txt"



dataset = load_dataset("json", data_files=ds_path, split="train")
dataset = dataset.shuffle(seed=42)
size = int(len(dataset)/3)
dataset = dataset.select(range(size))
with open(output_file, "a") as f:
    print(len(dataset))
    for line in tqdm(dataset):
        text = line["text"]
        f.write(text)
print("output: ", output_file)

42095


  0%|          | 0/42095 [00:00<?, ?it/s]

output:  /content/drive/MyDrive/geniac_tokenizer/merge_4category_tokenizer_4.0_val_one_third.txt


## train

In [ ]:
## train
output_base="/content/drive/MyDrive/geniac_tokenizer"
# ds_path="/content/drive/MyDrive/geniac_tokenizer/merge_5category_tokenizer_4.0.txt"
ds_path="/content/drive/MyDrive/geniac_tokenizer/merge_4category_tokenizer_4.0.txt"


import sentencepiece as spm
import os

def train(path: str, name: str, vocab_size: int = 20000, coverage: float = 0.995, model_type: str = 'bpe', max_sentence_length: int = 4096, byte_fallback=False):
    output_path=f"{output_base}/{name}"
    if not os.path.exists(output_path):
        print("created...", output_path)
        os.makedirs(output_path)
    model_prefix = f"{output_path}/tokenizer"
    print("start...", name)
    spm.SentencePieceTrainer.Train(
        input=path,
        model_prefix=model_prefix,
        vocab_size=vocab_size,
        character_coverage=coverage,
        model_type=model_type,
        max_sentence_length=max_sentence_length,
        byte_fallback=byte_fallback,
        train_extremely_large_corpus=True,
        input_sentence_size=7000000
    )
    spm.SentencePieceProcessor(model_file=f"{model_prefix}.model")
    print(model_prefix, "done...")

In [ ]:
## upload
from transformers import T5Tokenizer
import os
from transformers import AutoTokenizer


UNK_TOKEN = "<unk>"
BOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"
PAD_TOKEN = "<pad>"
CLS_TOKEN = "<CLS>"
SEP_TOKEN = "<SEP>"
EOD_TOKEN = "<EOD>"
MASK_TOKEN = "<MASK>"
NEWLINE_TOKEN = "\n"

def check(load_path):
  print(load_path)
  #text="あのイーハトーヴォのすきとおった風、夏でも底に冷たさをもつ青いそら、うつくしい森で飾られたモリーオ市"
  text="「吾輩は猫である。名前はまだ無い。」夏目漱石の『吾輩は猫である』の冒頭の一文である。"
  text="アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。"
  tokenizer = AutoTokenizer.from_pretrained(load_path, device_map="auto")
  t = tokenizer(text)
  d = tokenizer.decode(t.input_ids)
  print("decode:", d)
  print("-"*100)
  t = tokenizer.tokenize(text)
  print("tokenize: ", t)
  print("="*100)
  return tokenizer

def save_and_upload(input, output, repo):
  output_tokenizer = T5Tokenizer(
        vocab_file=input,
        bos_token=BOS_TOKEN,
        eos_token=EOS_TOKEN,
        unk_token=UNK_TOKEN,
        sep_token=SEP_TOKEN,
        pad_token=PAD_TOKEN,
        cls_token=CLS_TOKEN,
        mask_token=MASK_TOKEN,
        additional_special_tokens=[
            EOD_TOKEN,
        ],  # Note: `NEWLINE_TOKEN` is NOT needed in `additional_special_tokens`.
        extra_ids=0,
        model_max_length=4096,  # TODO: Remove hard coding and/or magic number.
        split_special_tokens=True,
        legacy=False
    )

  os.makedirs(output, exist_ok=True)
  output_tokenizer.save_pretrained(output)
  print("-"*100)
  tokenizer = check(output)
  tokenizer.push_to_hub(repo)
  print("push...", repo)
  check(repo)
  print("done...")
  print("="*100)

In [ ]:
## pattern1
name="unigram_4cat_50000"

train(
    ds_path,
    name,
    vocab_size=50000,
    model_type="unigram"
    )

## pattern1
base_path="/content/drive/MyDrive/geniac_tokenizer"
tokenizer_file=f"{base_path}/{name}/tokenizer.model"
output_tokenizer_dir=f"{base_path}/{name}"
repo_id=f"team-sanai/{name}"
save_and_upload(tokenizer_file, output_tokenizer_dir, repo_id)

----------------------------------------------------------------------------------------------------
/content/drive/MyDrive/geniac_tokenizer/unigram_4cat_50000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁アイ', 'ン', 'シュタイン', 'は', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表し', '、', '物理学', 'に', '革命', 'をもたらした', '。']


spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

push... team-sanai/unigram_4cat_50000
team-sanai/unigram_4cat_50000


...

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁アイ', 'ン', 'シュタイン', 'は', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表し', '、', '物理学', 'に', '革命', 'をもたらした', '。']
done...


In [ ]:
## pattern2
name="unigram_4cat_32000"

train(
    ds_path,
    name,
    vocab_size=32000,
    model_type="unigram"
    )

## pattern2
base_path="/content/drive/MyDrive/geniac_tokenizer"
tokenizer_file=f"{base_path}/{name}/tokenizer.model"
output_tokenizer_dir=f"{base_path}/{name}"
repo_id=f"team-sanai/{name}"
save_and_upload(tokenizer_file, output_tokenizer_dir, repo_id)

created... /content/drive/MyDrive/geniac_tokenizer/unigram_4cat_32000
start... unigram_4cat_32000
/content/drive/MyDrive/geniac_tokenizer/unigram_4cat_32000/tokenizer done...
----------------------------------------------------------------------------------------------------
/content/drive/MyDrive/geniac_tokenizer/unigram_4cat_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁', 'ア', 'イン', 'シュ', 'タイ', 'ンは', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたらし', 'た', '。']


spiece.model:   0%|          | 0.00/774k [00:00<?, ?B/s]

push... team-sanai/unigram_4cat_32000
team-sanai/unigram_4cat_32000


...

tokenizer.json:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁', 'ア', 'イン', 'シュ', 'タイ', 'ンは', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたらし', 'た', '。']
done...


In [ ]:
## pattern3
name="bpe_4cat_50000"

train(
    ds_path,
    name,
    vocab_size=50000,
    model_type="bpe"
    )

## pattern3
base_path="/content/drive/MyDrive/geniac_tokenizer"
tokenizer_file=f"{base_path}/{name}/tokenizer.model"
output_tokenizer_dir=f"{base_path}/{name}"
repo_id=f"team-sanai/{name}"
save_and_upload(tokenizer_file, output_tokenizer_dir, repo_id)

In [ ]:
## pattern4
name="bpe_4cat_32000"

train(
    ds_path,
    name,
    vocab_size=32000,
    model_type="bpe"
    )

base_path="/content/drive/MyDrive/geniac_tokenizer"
tokenizer_file=f"{base_path}/{name}/tokenizer.model"
output_tokenizer_dir=f"{base_path}/{name}"
repo_id=f"team-sanai/{name}"
save_and_upload(tokenizer_file, output_tokenizer_dir, repo_id)

created... /content/drive/MyDrive/geniac_tokenizer/bpe_4cat_32000
start... bpe_4cat_32000
/content/drive/MyDrive/geniac_tokenizer/bpe_4cat_32000/tokenizer done...
----------------------------------------------------------------------------------------------------
/content/drive/MyDrive/geniac_tokenizer/bpe_4cat_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁ア', 'イン', 'シュタ', 'イン', 'は', '、19', '05', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたら', 'した', '。']


spiece.model:   0%|          | 0.00/754k [00:00<?, ?B/s]

push... team-sanai/bpe_4cat_32000
team-sanai/bpe_4cat_32000


...

tokenizer.json:   0%|          | 0.00/2.02M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁ア', 'イン', 'シュタ', 'イン', 'は', '、19', '05', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたら', 'した', '。']
done...


In [ ]:
## pattern5
name="unigram_bf_4cat_50000"

train(
    ds_path,
    name,
    vocab_size=50000,
    model_type="unigram",
    byte_fallback=True
)

base_path="/content/drive/MyDrive/geniac_tokenizer"
tokenizer_file=f"{base_path}/{name}/tokenizer.model"
output_tokenizer_dir=f"{base_path}/{name}"
repo_id=f"team-sanai/{name}"
save_and_upload(tokenizer_file, output_tokenizer_dir, repo_id)

created... /content/drive/MyDrive/geniac_tokenizer/unigram_bf_4cat_50000
start... unigram_bf_4cat_50000
/content/drive/MyDrive/geniac_tokenizer/unigram_bf_4cat_50000/tokenizer done...
----------------------------------------------------------------------------------------------------
/content/drive/MyDrive/geniac_tokenizer/unigram_bf_4cat_50000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁アイ', 'ン', 'シュタイン', 'は', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表し', '、', '物理学', 'に', '革命', 'をもたらした', '。']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁アイ', 'ン', 'シュタイン', 'は', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表し', '、', '物理学', 'に', '革命', 'をもたらした', '。']
done...


In [ ]:
## pattern6
name="unigram_bf_4cat_32000"

train(
    ds_path,
    name,
    vocab_size=32000,
    model_type="unigram",
    byte_fallback=True
    )


base_path="/content/drive/MyDrive/geniac_tokenizer"
tokenizer_file=f"{base_path}/{name}/tokenizer.model"
output_tokenizer_dir=f"{base_path}/{name}"
repo_id=f"team-sanai/{name}"
save_and_upload(tokenizer_file, output_tokenizer_dir, repo_id)

created... /content/drive/MyDrive/geniac_tokenizer/unigram_bf_4cat_32000
start... unigram_bf_4cat_32000
/content/drive/MyDrive/geniac_tokenizer/unigram_bf_4cat_32000/tokenizer done...
----------------------------------------------------------------------------------------------------
/content/drive/MyDrive/geniac_tokenizer/unigram_bf_4cat_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁', 'ア', 'イン', 'シュ', 'タイ', 'ンは', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたらし', 'た', '。']


spiece.model:   0%|          | 0.00/773k [00:00<?, ?B/s]

push... team-sanai/unigram_bf_4cat_32000
team-sanai/unigram_bf_4cat_32000


...

tokenizer.json:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁', 'ア', 'イン', 'シュ', 'タイ', 'ンは', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたらし', 'た', '。']
done...


In [ ]:
check("team-sanai/unigram_50000")
print()
print()
check("team-sanai/unigram_32000")
print()
print()
#heck("team-sanai/bpe_50000")
#print()
#print()
check("team-sanai/bpe_32000")
print()
print()
check("team-sanai/unigram_bf_50000")
print()
print()
check("team-sanai/unigram_bf_32000")

team-sanai/unigram_50000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: 「<unk>は猫である。名前はまだ無い。」夏目<unk>石の『<unk>は猫である』の<unk>頭の一文である。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁「', '吾輩', 'は', '猫', 'である', '。', '名前', 'はまだ', '無い', '。」', '夏', '目', '漱', '石の', '『', '吾輩', 'は', '猫', 'である', '』', 'の', '冒', '頭の', '一', '文', 'である', '。']
team-sanai/unigram_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: 「<unk>は猫である。名前はまだ無い。」夏目<unk>石の『<unk>は猫である』の<unk>頭の一文である。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁「', '吾輩', 'は', '猫', 'である', '。', '名前', 'はまだ', '無い', '。」', '夏', '目', '漱', '石の', '『', '吾輩', 'は', '猫', 'である', '』', 'の', '冒', '頭の', '一', '文', 'である', '。']
team-sanai/bpe_50000


...

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: 「<unk>は猫である。名前はまだ無い。」夏目<unk>石の『<unk>は猫である』の<unk>頭の一文である。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁「', '吾輩', 'は', '猫', 'である', '。', '名前', 'はまだ', '無い', '。」', '夏', '目', '漱', '石の', '『', '吾輩', 'は', '猫', 'である', '』', 'の', '冒', '頭の', '一', '文', 'である', '。']
team-sanai/unigram_bf_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: 「<unk>は猫である。名前はまだ無い。」夏目<unk>石の『<unk>は猫である』の<unk>頭の一文である。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁「', '吾輩', 'は', '猫', 'である', '。', '名前', 'はまだ', '無い', '。」', '夏', '目', '漱', '石の', '『', '吾輩', 'は', '猫', 'である', '』', 'の', '冒', '頭の', '一', '文', 'である', '。']


T5TokenizerFast(name_or_path='team-sanai/unigram_bf_32000', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<SEP>', 'pad_token': '<pad>', 'cls_token': '<CLS>', 'mask_token': '<MASK>', 'additional_special_tokens': ['<EOD>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<SEP>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<CLS>", rstrip=False, l

## eval

In [ ]:
from transformers import AutoTokenizer
# from tqdm import tqdm
from tqdm.notebook import tqdm

def batch(x, batch_size, drop_last=False):
    bs = batch_size
    length = len(x)
    if drop_last or length % bs == 0:
        len_batch = int(length / bs)
    else:
        len_batch = int(length / bs) + 1

    for idx in range(len_batch):
        yield x[idx * bs : (idx + 1) * bs]

def evaluate_tokenizer(tokenizer_id, eval_corpus_text):
    print("Evaluating tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, device_map="auto")

    print(f"length of texts: {len(eval_corpus_text)}")
    batch_len = 10000
    num_batches = (len(eval_corpus_text) - 1) // batch_len + 1
    print(f"num of batches: {num_batches}")

    txt_len = 0
    total_tokens = 0
    unk_count = 0
    # unk_token_id = tokenizer.unk_id()
    unk_token_id = tokenizer.unk_token_id

    error_words = []
    for text in tqdm(batch(eval_corpus_text, batch_len), total=num_batches):
        encoded = tokenizer.encode(text)
        decoded = tokenizer.decode(encoded)
        txt_len += len(text)
        unk_count += encoded.count(unk_token_id)
        total_tokens += len(encoded)
        for word in text.split():
            if word not in decoded:
                error_words.append(word)

    unk_rate = unk_count / total_tokens
    print(f"Unknown word rate: {unk_rate}")

    # 平均トークン長を計算
    avg_len = total_tokens / txt_len
    print(f"Average token length: {avg_len}")

    # トークン化にかかる時間を計測 (ダミーの値を使用)
    speed = 1000  # トークン/秒
    print(f"Tokenization speed: {speed} tokens/sec")

    return {
        'unk_rate': unk_rate,
        'avg_len': avg_len,
        'speed': speed,
        'error_word_count': len(error_words),
        'error_words': error_words,
    }

    print(result)
    print("="*200)
    return result

In [ ]:
data_path="/content/drive/MyDrive/geniac_tokenizer/merge_4category_tokenizer_4.0_val.txt"
data_path="/content/drive/MyDrive/geniac_tokenizer/merge_4category_tokenizer_4.0_val_one_third.txt"

with open(data_path, 'r') as f:
  content = f.read()

evaluate_tokenizer("team-sanai/unigram_4cat_50000", content)
evaluate_tokenizer("team-sanai/unigram_4cat_32000", content)
# run_eval("team-sanai/bpe_4cat_50000", dataset)
evaluate_tokenizer("team-sanai/bpe_4cat_32000", content)
evaluate_tokenizer("team-sanai/unigram_bf_4cat_50000", content)
evaluate_tokenizer("team-sanai/unigram_bf_4cat_32000", content)
print()

Evaluating tokenizer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


length of texts: 95202228
num of batches: 9521


  0%|          | 0/9521 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4960 > 4096). Running this sequence through the model will result in indexing errors


...

length of texts: 95202228
num of batches: 9521


  0%|          | 0/9521 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4190 > 4096). Running this sequence through the model will result in indexing errors


Unknown word rate: 0.009943735360028308
Average token length: 0.3150161464708578
Tokenization speed: 1000 tokens/sec


{'unk_rate': 0.009943735360028308,
 'avg_len': 0.3150161464708578,
 'speed': 1000,
 'error_word_count': 194023,
 'error_words': ['son.質問:ジェームズは、2つのエピソードの危険とホイールオブフォーチュンのエピソードを見ました。危険は20分で、ホイールオブフォーチュンは2倍の長さです。彼は何時間テレビを見ましたか？回答:Pythonコードを使用してこの問題を解決しましょう。jeopardy_duration',
  '{2（x+1）}',
  '{3（$$$2-$$$3x）}',
  '{align*}質問:バイナリ表現に6桁を必要とする最小のベース10の数値の値はいくらですか？回答:数のバイナリ表現を計算する関数を作成しましょう。def',
  'bin(number)[2:]print(binary(21))10101次に、6桁のバイナリ表現を備えた最小の整数を見つけましょう。smallest_6_digit',
  'AFP】国際テロ組織アルカイダ（Al-Qaeda）の最高指導者だったウサマ・ビンラディン（Osama',
  'Laden）容疑者を潜伏先のパキスタンで急襲した米軍の作戦で、ビンラディン容疑者を殺害した米海軍特殊部隊（ネイビーシールズ、Navy',
  'SEALs）の元隊員が沈黙を破り、3発の銃弾を撃ち込んだ瞬間についてインタビューで語った。現在は失業中で、直面している経済的な不安などについても触れている。',
  'シールズ元隊員は自分の身元については伏せたまま、米エスクァイア（Esquire）誌のインタビューに応え、2011年5月の急襲作戦で自分が果たした役割を初めて明かした。隊員はビンラディン容疑者を目にした印象について「混乱しているように見えた。そして思っていたよりもずっと背が高かった」と語っている。',
  'パキスタン北部アボタバード（Abbottabad）にあった隠れ家の3階に作戦部隊が足を踏み入れると、暗闇の中にビンラディン容疑者がいた。ビンラディン容疑者は年齢の一番若い妻の肩に両手をかけ、「前に向かって彼女を押すよう」な行動をとったという。近くにはカラシニコフ自動小銃（AK-47）があった。',


In [ ]:
evaluate_tokenizer("llm-jp/llm-jp-13b-v1.0", content)

evaluate_tokenizer("mistralai/Mixtral-8x7B-v0.1", content)
print()

Evaluating tokenizer...


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

length of texts: 95202228
num of batches: 9521


...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

length of texts: 95202228
num of batches: 9521


  0%|          | 0/9521 [00:00<?, ?it/s]

Unknown word rate: 0.0
Average token length: 0.4450233454620411
Tokenization speed: 1000 tokens/sec



In [ ]:
check("team-sanai/unigram_4cat_50000")
print()
print()
check("team-sanai/unigram_4cat_32000")
print()
print()
#heck("team-sanai/bpe_4cat_50000")
#print()
#print()
check("team-sanai/bpe_4cat_32000")
print()
print()
check("team-sanai/unigram_bf_4cat_50000")
print()
print()
check("team-sanai/unigram_bf_4cat_32000")

team-sanai/unigram_4cat_50000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁アイ', 'ン', 'シュタイン', 'は', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表し', '、', '物理学', 'に', '革命', 'をもたらした', '。']


team-sanai/unigram_4cat_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁', 'ア', 'イン', 'シュ', 'タイ', 'ンは', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたらし', 'た', '。']


team-sanai/bpe_4cat_32000


...

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁アイ', 'ン', 'シュタイン', 'は', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表し', '、', '物理学', 'に', '革命', 'をもたらした', '。']


team-sanai/unigram_bf_4cat_32000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


decode: アインシュタインは、1905年に特殊相対性理論を発表し、物理学に革命をもたらした。</s>
----------------------------------------------------------------------------------------------------
tokenize:  ['▁', 'ア', 'イン', 'シュ', 'タイ', 'ンは', '、1', '905', '年に', '特殊', '相対', '性', '理論', 'を発表', 'し', '、', '物理学', 'に', '革命', 'をもたらし', 'た', '。']


T5TokenizerFast(name_or_path='team-sanai/unigram_bf_4cat_32000', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<SEP>', 'pad_token': '<pad>', 'cls_token': '<CLS>', 'mask_token': '<MASK>', 'additional_special_tokens': ['<EOD>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<SEP>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<CLS>", rstrip=Fal